In [51]:
import pandas as pd
import sqlite3
import src.data_enrichment as enrich

import geopandas as gpd

In [52]:
conn = sqlite3.connect("housing_info.db")
df_master = pd.read_sql_query('SELECT * FROM df_master', conn)

t_geodata = gpd.read_file('opendata/local-area-boundary.shp')
t_geodata['geometry'] = t_geodata['geometry'].to_crs({'init': 'EPSG:4269'})

C:\Users\Alan\anaconda3\envs\housing_project\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [53]:
coords_mapping = pd.read_csv('data/coords_mapping.csv').dropna()

In [55]:
import json
def convert_to_json(data_string):
    # Removing comments
    data_string = ''.join(i for i in data_string.splitlines(True) if not i.strip().startswith('#'))

    # Removing '\n' characters
    data_string = data_string.replace('\n', '')

    # Load as JSON
    json_data = json.loads(data_string)

    return json_data

coords_mapping['JSON'] = coords_mapping['JSON'].apply(convert_to_json)

In [56]:
from shapely.geometry import Polygon

def create_polygon(json):
    coordinates = [(coords["lng"], coords["lat"]) for coords in json]
    polygon = Polygon(coordinates)

    return polygon

coords_mapping['polygon'] = coords_mapping['JSON'].apply(create_polygon)

coords_df = pd.DataFrame({'name': coords_mapping.Neighborhood,
                          'geometry': coords_mapping.polygon})
t_geodata = pd.concat([t_geodata,coords_df], axis = 0).reset_index(drop = True)

In [57]:
import json
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar

#Read data to json.
t_geodata_json = json.loads(t_geodata.to_json())
#Convert to String like object.
json_data = json.dumps(t_geodata_json)

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

p = figure(title = 'GVA neighbourhoods', plot_height = 600 , plot_width = 950)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,
          line_color = 'white', line_width = 0.25, fill_alpha = 1)

#Display figure in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)

Loading BokehJS ...

asdf


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/TableMatchingCriteria?GeographyType=MetropolitanMajorArea&GeographyId=2410&CategoryLevel1=Population%2C%20Households%20and%20Housing%20Stock&CategoryLevel2=Household%20Income&ColumnField=HouseholdIncomeRange&RowField=Neighbourhood&SearchTags%5B0%5D.Key=Households&SearchTags%5B0%5D.Value=Number&SearchTags%5B1%5D.Key=Statistics&SearchTags%5B1%5D.Value=AverageAndMedian"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table element
    table = soup.find('table')
    
    # Extract table headers
    headers = [header.text.strip() for header in table.find_all('th')]
    
    # Extract table rows
    rows = []
    for row in table.find_all('tr'):
        rows.append([cell.text.strip() for cell in row.find_all(['td', 'th'])])  # Include th for the first column

else:
    print("Failed to retrieve data. Status code:", response.status_code)

In [ ]:
income_df = pd.DataFrame(rows)

income_df[0][0] = 'City'
new_header = income_df.iloc[0] #grab the first row for the header
income_df = income_df[1:] #take the data less the header row
income_df.columns = new_header #set the header row as the df header

income_df.to_csv('data/income_df.csv')

In [ ]:
income_df